In [1]:
import os 

In [2]:
%pwd

'/Users/sunilverma/Text-Summarizer-Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/sunilverma/Text-Summarizer-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig: 
    root_dir: Path 
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [7]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        # Convert relative paths to absolute
        if not config_filepath.is_absolute():
            config_filepath = Path(os.getcwd()) / config_filepath
        if not params_filepath.is_absolute():
            params_filepath = Path(os.getcwd()) / params_filepath
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir]) 
        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name,
        )
        return model_evaluation_config

In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Split the dataset into batch-sized chunks."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(
        self,
        dataset,
        metric,
        model,
        tokenizer,
        batch_size=16,
        device="cuda" if torch.cuda.is_available() else "cpu",
        column_text="article",
        column_summary="highlights",
        gen_kwargs=None,
    ):
        if gen_kwargs is None:
            gen_kwargs = {"length_penalty": 0.8, "num_beams": 2, "max_length": 64}

        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)
        ):
            inputs = tokenizer(
                article_batch,
                max_length=1024,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            with torch.no_grad():
                summaries = model.generate(
                    input_ids=inputs["input_ids"].to(device),
                    attention_mask=inputs["attention_mask"].to(device),
                    **gen_kwargs,
                )

            decoded_summaries = [
                tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                for s in summaries
            ]
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = evaluate.load("rouge")

        eval_sample_size = 5
        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt["test"][:eval_sample_size],
            rouge_metric,
            model_pegasus,
            tokenizer,
            batch_size=2,
            device=device,
            column_text="dialogue",
            column_summary="summary",
        )

        rouge_dict = {}
        for rn in rouge_names:
            val = score[rn]
            if hasattr(val, "mid"):
                val = val.mid.fmeasure
            rouge_dict[rn] = float(val)

        df = pd.DataFrame(rouge_dict, index=["pegasus"])
        df.to_csv(self.config.metric_file_name, index=False)
        return df

In [25]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

2025-12-19 20:17:19,406: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/config/config.yaml loaded successfully
2025-12-19 20:17:19,417: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/params.yaml loaded successfully
2025-12-19 20:17:19,421: INFO: common: created directory at: artifacts
2025-12-19 20:17:19,424: INFO: common: created directory at: artifacts/model_evaluation


100%|██████████| 5/5 [30:44<00:00, 368.89s/it]


2025-12-19 20:48:07,989: INFO: rouge_scorer: Using default tokenizer.


AttributeError: 'numpy.float64' object has no attribute 'mid'